### Setup

#### Instalación de paquetes

In [1]:
# version de transformers para usar PPLM
!pip install transformers==3.4.0 torch==1.7.0 nlp colorama==0.4.4

     |████████████████████████████████| 1.3MB 2.6MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 1.7MB 41.1MB/s 
     |████████████████████████████████| 870kB 8.6MB/s 
     |████████████████████████████████| 1.2MB 36.0MB/s 
     |████████████████████████████████| 2.9MB 35.7MB/s 
     |████████████████████████████████| 245kB 44.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=67ba8bcbb11d99a71918dc37ad58ef6d70910a4d780f3851409ba6880d376999
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Succe

#### Imports

In [2]:
import torch, os, re, pandas as pd, json, numpy as np, json
from sklearn.model_selection import train_test_split
from transformers import TextDataset, DataCollatorForLanguageModeling, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

In [3]:
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = None

#### Conexión con drive

In [4]:
# apuntar a google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
os.chdir('/content/drive/MyDrive/spainai_webinar')

### Carga de GPT2 y tokenizer

In [7]:
device = "cuda:0"

In [8]:
# opciones: ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
base_model = GPT2LMHeadModel.from_pretrained('gpt2')
base_model = base_model.to(device)
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

### Finetuning headlines

#### Carga de datos y preprocesamiento

##### The guardian financial headlines

In [9]:
guardian_path= './data/guardian_headlines.csv'
guardian_df = pd.read_csv(guardian_path, encoding = 'utf-8')

In [10]:
guardian_df.shape

(17800, 2)

In [11]:
guardian_df.head(10)

,Time,Headlines
0,18-Jul-20,Johnson is asking Santa for a Christmas recovery
1,18-Jul-20,‘I now fear the worst’: four grim tales of working life upended by Covid-19
2,18-Jul-20,Five key areas Sunak must tackle to serve up economic recovery
3,18-Jul-20,Covid-19 leaves firms ‘fatally ill-prepared’ for no-deal Brexit
4,18-Jul-20,The Week in Patriarchy \n\n\n Bacardi's 'lady vodka': the latest in a long line of depressing gendered products
5,18-Jul-20,English councils call for smoking ban outside pubs and cafes
6,18-Jul-20,Can Tesla justify a $300bn valuation?
7,18-Jul-20,Empty city centres: 'I’m not sure it will ever be the same again'
8,18-Jul-20,Democratising finance for all? An investment app for amateurs and a student trader's death
9,18-Jul-20,"Homebuyer loses £300,000 to fraudsters – but gets it back after we step in"


In [12]:
guardian_df['Headlines'].str.split().apply(lambda x: len(x)).describe()

count    17800.000000
mean        11.021348
std          2.667269
min          3.000000
25%          9.000000
50%         11.000000
75%         12.000000
max         30.000000
Name: Headlines, dtype: float64

In [13]:
guardian_df['Headlines'].str.split().apply(lambda x: len(x)).quantile(np.linspace(.1, 1, 18, 0), 'lower')

0.10     8
0.15     9
0.20     9
0.25     9
0.30    10
0.35    10
0.40    10
0.45    10
0.50    11
0.55    11
0.60    11
0.65    12
0.70    12
0.75    12
0.80    13
0.85    14
0.90    15
0.95    16
Name: Headlines, dtype: int64

In [14]:
# Quitamos filas vacias o a null
titulo_vacio = (guardian_df['Headlines'].str.len() == 0) | guardian_df['Headlines'].isna()
guardian_df = guardian_df[~titulo_vacio]

# Quitamos headlines con menos de 8 palabras
guardian_df = guardian_df[(guardian_df['Headlines'].str.split().apply(lambda x: len(x)) >= 8)]

In [15]:
len(guardian_df)

16807

##### Sports headlines

In [16]:
tagged_news_filepath = './data/News_Category_Dataset_v2.json'
news = []
for line in open(tagged_news_filepath, 'r'):
    news.append(json.loads(line))

tagged_news_df = pd.DataFrame(news)

In [17]:
len(tagged_news_df)

200853

In [18]:
tagged_news_df.head(10)

,category,headline,authors,link,short_description,date
0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,She left her husband. He killed their children. Just another day in America.,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Andy McDonald,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,"The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment.""",2018-05-26
5,ENTERTAINMENT,Morgan Freeman 'Devastated' That Sexual Harassment Claims Could Undermine Legacy,Ron Dicker,https://www.huffingtonpost.com/entry/morgan-freeman-devastated-sexual-misconduct_us_5b096319e4b0802d69cba298,"""It is not right to equate horrific incidents of sexual assault with misplaced compliments or humor,"" he said in a statement.",2018-05-26
6,ENTERTAINMENT,Donald Trump Is Lovin' New McDonald's Jingle In 'Tonight Show' Bit,Ron Dicker,https://www.huffingtonpost.com/entry/donald-trump-mcondalds-tonight-show_us_5b093561e4b0fdb2aa53daba,"It's catchy, all right.",2018-05-26
7,ENTERTAINMENT,What To Watch On Amazon Prime That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/amazon-prime-what-to-watch_us_5b044625e4b0c0b8b23ec14f,There's a great mini-series joining this week.,2018-05-26
8,ENTERTAINMENT,Mike Myers Reveals He'd 'Like To' Do A Fourth Austin Powers Film,Andy McDonald,https://www.huffingtonpost.com/entry/mike-myers-reveals-he-wants-to-do-a-fourth-austin-powers-film_us_5b096198e4b0802d69cb9f15,"Myer's kids may be pushing for a new ""Powers"" film more than anyone.",2018-05-26
9,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what-to-watch_us_5b0445bae4b0c0b8b23ec046,You're getting a recent Academy Award-winning movie.,2018-05-26


In [19]:
tagged_news_df['category'].unique()

array(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION',
       'SCIENCE', 'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTS',
       'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE', 'HEALTHY LIVING',
       'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST', 'FIFTY', 'ARTS',
       'WELLNESS', 'PARENTING', 'HOME & LIVING', 'STYLE & BEAUTY',
       'DIVORCE', 'WEDDINGS', 'FOOD & DRINK', 'MONEY', 'ENVIRONMENT',
       'CULTURE & ARTS'], dtype=object)

In [20]:
sports_news = tagged_news_df[tagged_news_df['category'] == 'SPORTS']

In [21]:
len(sports_news)

4884

In [22]:
sports_news.head(10)

,category,headline,authors,link,short_description,date
80,SPORTS,Jets Chairman Christopher Johnson Won't Fine Players For Anthem Protests,Ron Dicker,https://www.huffingtonpost.com/entry/jets-christopher-johnson-wont-fine-players-for-anthem-protests_us_5b069e29e4b05f0fc8453d6f,"“I never want to put restrictions on the speech of our players.""",2018-05-24
101,SPORTS,Trump Posthumously Pardons Boxer Jack Johnson,Lydia O'Connor,https://www.huffingtonpost.com/entry/trump-pardons-late-boxing-champion-jack-johnson_us_5b06eea4e4b07c4ea10630c8,The pardoning of the black heavyweight boxer is only the third posthumous pardon in U.S. history.,2018-05-24
135,SPORTS,Anna Kournikova Dancing With Her Bouncing Baby Is The Cutest Thing,Ron Dicker,https://www.huffingtonpost.com/entry/anna-kournikova-dancing-baby_us_5b06d0c9e4b0784cd2b25d63,The score is love-love.,2018-05-24
136,SPORTS,Trump Says NFL Players Unwilling To Stand For Anthem Maybe 'Shouldn't Be In The Country',Doha Madani and Willa Frej,https://www.huffingtonpost.com/entry/trump-nfl-kneeling-anthem-rule_us_5b05cbf4e4b05f0fc84445c6,"""I don’t think people should be staying in locker rooms,"" Trump told Fox News. ""You have to stand proudly for the national anthem or you shouldn’t be playing.""",2018-05-24
154,SPORTS,Brandi Chastain Totally Agrees Her Hall Of Fame Plaque Looks Nothing Like Her,Lee Moran,https://www.huffingtonpost.com/entry/brandi-chastain-plaque-odd_us_5b052506e4b0784cd2afb267,Was it supposed to be Gary Busey or Mickey Rooney?,2018-05-23
186,SPORTS,NFL Bans Kneeling During The National Anthem,Ron Dicker,https://www.huffingtonpost.com/entry/nfl-penalty-kneeling-national-anthem_us_5b052eb7e4b05f0fc8431d64,Teams can set their own policies for punishing players.,2018-05-23
358,SPORTS,The History Of The National Anthem In Sports,Caroline Bologna,https://www.huffingtonpost.com/entry/history-national-anthem-sports_us_5afc9bcfe4b06a3fb50d5056,"How ""The Star-Spangled Banner"" became a part of American sports culture.",2018-05-19
631,SPORTS,Israeli Soccer Team Beitar Jerusalem To Add 'Trump' To Its Name,Ron Dicker,https://www.huffingtonpost.com/entry/israeli-soccer-team-says-its-adding-trump-to-its-name_us_5af95f74e4b0e57cd9fb49c7,Beitar Jerusalem said it's making the move to honor the U.S. embassy's relocation.,2018-05-14
645,SPORTS,"Coach Tells Hurt UFC Fighter To Continue After She Says, 'I'm Done'",Ron Dicker,https://www.huffingtonpost.com/entry/ufc-raquel-pennington-amanda-nunes_us_5af99e9ee4b032b10bfd142d,Raquel Pennington took even more of a beating in a bloody loss to champion Amanda Nunes.,2018-05-14
951,SPORTS,$18 Derby Day Bet Wins Thrilled Texas Woman $1.23 Million,Mary Papenfuss,https://www.huffingtonpost.com/entry/texan-wins-big-with-kentucky-derby-day-bet_us_5af108a9e4b0c4f193260254,"It was like a ""mini-lotto,"" says the Pick 5 winner.",2018-05-08


In [23]:
sports_news['headline'].str.split().apply(lambda x: len(x)).describe()

count    4884.000000
mean       10.231368
std         2.692004
min         1.000000
25%         9.000000
50%        10.000000
75%        12.000000
max        26.000000
Name: headline, dtype: float64

In [24]:
sports_news['headline'].str.split().apply(lambda x: len(x)).quantile(np.linspace(.05, 1, 19, 0), 'lower')

0.05     6
0.10     7
0.15     8
0.20     8
0.25     9
0.30     9
0.35     9
0.40    10
0.45    10
0.50    10
0.55    11
0.60    11
0.65    11
0.70    12
0.75    12
0.80    12
0.85    13
0.90    13
0.95    14
Name: headline, dtype: int64

In [25]:
# Quitamos filas vacias o a null
titulo_vacio = (sports_news['headline'].str.len() == 0) | sports_news['headline'].isna()
sports_news = sports_news[~titulo_vacio]

# Quitamos headlines con menos de 8 palabras
sports_news = sports_news[(sports_news['headline'].str.split().apply(lambda x: len(x)) >= 8)]

In [26]:
len(sports_news)

4162

##### Articles dataset

In [27]:
def remove_publication_headline(headline, publication):
  # publication col doesn't match exactly with newspaper in title col
  if publication in headline:
    headline = headline.split(' - ')[0]
  return headline

In [28]:
filepath= './data/articles1.csv'
filepath2 = './data/articles2.csv'
filepath3 = './data/articles3.csv'

news_df1 = pd.read_csv(filepath, encoding = 'utf-8')
news_df2 = pd.read_csv(filepath2, encoding = 'utf-8')
news_df3 = pd.read_csv(filepath3, encoding = 'utf-8')

In [29]:
news_df = news_df1.append(news_df2).append(news_df3)

In [30]:
news_df['year'].unique()

array([2016., 2017., 2014., 2015., 2013., 2012., 2011.,   nan, 2004.,
       2008., 2003., 2007., 2005., 2010., 2009., 2000.])

In [31]:
len(news_df)

142570

In [32]:
# Quitamos filas vacias o a null
titulo_vacio = (news_df['title'].str.len() == 0) | news_df['title'].isna()
contenido_vacio = (news_df['content'].str.len() == 0) | news_df['content'].isna()
news_df = news_df[~titulo_vacio & ~contenido_vacio]

# Quitamos publication del title
news_df['title'] = news_df.apply(lambda row: remove_publication_headline(row['title'], row['publication']), axis = 1)

# Quitamos headlines con menos de 8 palabras
news_df = news_df[(news_df['title'].str.split().apply(lambda x: len(x)) >= 8)]

In [33]:
len(news_df)

116522

In [34]:
news_df['title'].str.split().apply(lambda x: len(x)).describe()

count    116522.000000
mean         10.970727
std           2.457918
min           8.000000
25%           9.000000
50%          11.000000
75%          12.000000
max          33.000000
Name: title, dtype: float64

In [35]:
news_df['title'].str.split().apply(lambda x: len(x)).quantile(np.linspace(.05, 1, 19, 0), 'lower')

0.05     8
0.10     8
0.15     9
0.20     9
0.25     9
0.30     9
0.35    10
0.40    10
0.45    10
0.50    11
0.55    11
0.60    11
0.65    11
0.70    12
0.75    12
0.80    13
0.85    13
0.90    14
0.95    16
Name: title, dtype: int64

In [36]:
news_df['publication'].value_counts()

Breitbart              21148
New York Post          14140
NPR                    10762
Washington Post         9937
Reuters                 9917
Guardian                8304
New York Times          6738
Business Insider        6434
CNN                     6063
Vox                     4559
Buzzfeed News           4456
Talking Points Memo     3987
Fox News                3907
Atlantic                3311
National Review         2859
Name: publication, dtype: int64

In [37]:
news_df['title'].sample(10)

20538                                     Wikileaks: Hillary Clinton Ditched CGI Meeting Despite $12 Million from Morrocan King
35894                                                                       House hunting by helicopter takes off among Chinese
15818                                                           GOP Sen.: Refusal To Meet With SCOTUS Nominee Is ’Close-Minded’
25372                                             Third-man Fillon seen winning French primaries debate days before vote - poll
20658    Exclusive: 100+ Members of Special Forces Community Deliver Letter of Endorsement to Trump on 15th Anniversary of 9/11
15484                                                            How Will Rex Tillerson Explain Exxon Mobil’s ’Foreign Policy’?
30257                                               The Nuclear Option: Founders Smiling on Trump for Returning Power to People
10207                                                  Demographics Show Why Clinton, Trump Are Favored 

#### Generación del txt

In [38]:
def get_text_col_df(df, text_colname):
    text_df = df\
        [df[text_colname].notnull()]\
            .drop_duplicates(subset = [text_colname])\
                [[text_colname]]

    return text_df

def build_text_dataset(df, text_col, dest_path, eos = '<|endoftext|>', bos = '<|endoftext|>', sep_lines=False):
    f = open(dest_path, 'w', encoding = 'utf-8')
    data = ''
    lines = df[text_col].tolist()
    for line in lines:
        line = str(line).strip()
        line = re.sub(r"\s", " ", line)
        # line = re.sub(' +', ' ', line)
        
        if sep_lines == True:
            data += bos + ' ' + line + ' ' + eos + '\n'
        else:
            data += bos + ' ' + line + ' ' + eos

    f.write(data)

In [39]:
# headlines del dataset de articles
headlines_df = get_text_col_df(news_df[0:80000], 'title')

# headlines financieros de the guardian
headlines_guardian_df = get_text_col_df(guardian_df, 'Headlines').rename(columns = {'Headlines': 'title'})

# headlines de deportes (duplicados)
headlines_sports_df = get_text_col_df(sports_news, 'headline').rename(columns = {'headline': 'title'})

# se hace el append
headlines_train_df = headlines_df.append(headlines_sports_df).append(headlines_guardian_df)

In [40]:
headlines_train_df.shape

(100679, 1)

In [41]:
# se definen los eos y bos tokens
bos = '<|endoftext|>'
eos = '<|EOS|>'

In [42]:
train_guardian_path = './data/train_headlines_mix_articles.txt'
build_text_dataset(df=headlines_train_df, text_col = 'title', dest_path = train_guardian_path, bos=bos, eos = eos)

#### Entrenamiento

In [ ]:
# se añade el nuevo token al tokenizer y se redimensionan los embeddings del modelo
special_tokens_dict = {'eos_token': eos}
num_added_toks = base_tokenizer.add_special_tokens(special_tokens_dict)
base_model.resize_token_embeddings(len(base_tokenizer))

Embedding(50258, 768)

In [ ]:
model_path = './model_headlines_news_old_transformers'

In [ ]:
train_guardian_path

'./data/train_headlines_mix_articles.txt'

In [ ]:
train_dataset = TextDataset(
          tokenizer=base_tokenizer,
          file_path=train_guardian_path,
          block_size=128)

#validation_dataset = TextDataset(
#          tokenizer=base_tokenizer,
#          file_path=validation_path,
#          block_size=128)

data_collator = DataCollatorForLanguageModeling(
        tokenizer=base_tokenizer,
        mlm=False
    )

training_args = TrainingArguments(
    output_dir=model_path,
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=200,
    save_steps = 100000,
    weight_decay=0.01,
    logging_dir=model_path,
)

trainer = Trainer(
    model=base_model,
    args=training_args,
    data_collator = data_collator,
    train_dataset=train_dataset,
    prediction_loss_only=True
)

#eval_dataset=validation_dataset

/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:263: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


In [ ]:
trainer.train()

#### Guardado de modelo y tokenizer

In [ ]:
# se guarda el modelo y el tokenizer
trainer.save_model()
base_tokenizer.save_pretrained(model_path)

('./model_headlines_news_old_transformers/tokenizer_config.json',
 './model_headlines_news_old_transformers/special_tokens_map.json',
 './model_headlines_news_old_transformers/vocab.json',
 './model_headlines_news_old_transformers/merges.txt',
 './model_headlines_news_old_transformers/added_tokens.json')

In [ ]:
#del base_model
#del base_tokenizer

#### Carga de modelo y tokenizer

In [ ]:
model_path = './model_headlines_news_old_transformers'

In [ ]:
# carga de modelos entrenados
model = GPT2LMHeadModel.from_pretrained(model_path)
model = model.to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

#### Generación de texto con modelo entrenado

In [ ]:
def generate_n_text_samples(model, tokenizer, input_text, device, n_samples = 5):
  text_ids = tokenizer.encode(input_text, return_tensors = 'pt')
  text_ids = text_ids.to(device)

  generated_text_samples = model.generate(
      text_ids, 
      max_length= 100,  
      num_return_sequences= n_samples,
      no_repeat_ngram_size= 2,
      repetition_penalty= 1.5,
      top_p= 0.92,
      temperature= .85,
      do_sample= True,
      top_k= 125,
      early_stopping= True
  )

  return generated_text_samples


def print_generated_samples(generated_samples, tokenizer):
  for i, beam in enumerate(generated_samples):
    print(f"{i}: {tokenizer.decode(beam, skip_special_tokens=True)}")
    print()

# print text samples to first eos token
def print_text_samples_to_eos(generated_text_samples, tokenizer, input_ids, eos, bos, skip_special_tokens = False):
    for i, beam in enumerate(generated_text_samples):
        # decode text
        text = tokenizer.decode(beam, skip_special_tokens=skip_special_tokens)
        len_prompt = len(input_sentence)

        if bos in text[:len_prompt]:
            # skip bos
            text = text[len_prompt: text.find(eos)]
        
        else:
            # print everything
            text = text[: text.find(eos)]
        print("{}: {}".format(i, text))
        print()

In [ ]:
input_sentence = bos + ' '
generated_text_samples = generate_n_text_samples(model, tokenizer, input_sentence, device, n_samples = 10)

#Print output for each sequence generated above
print_text_samples_to_eos(generated_text_samples, tokenizer, input_sentence, eos, bos)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


0:  How long will Tesla Motors keep running? 

1:  'We're not interested in this.' The Guardian view on Trump’s NATO threat: ‘I know we are. We just can't be complacent' 

2:  A $400,000 Per Year Graduate’s Scholarship at Harvard 

3:  The U.S.-China Relationship Is the Best Chance for New Leaders 

4:  What a week! All eyes on the US election, as it happened 

5:  The US-Saudi relationship has become a haven for radical Islam in the Middle East 

6:  The US government has a new name for its ’bathroom law‘ 

7:  Why so many people are eating less water than they should — and why it's hard to count on this trend 

8:  The Trump-Russia inquiry is a political ploy by the left. They are not going to stop it 

9:  Stunning photo of China's first ever flag flies at U.S.-China World Cup 



### PPLM (Plug and Play Language Models)

In [ ]:
# para clonar el repo
#!git clone https://github.com/uber-research/PPLM.git

Cloning into 'PPLM'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 276 (delta 11), reused 1 (delta 0), pack-reused 250
Receiving objects: 100% (276/276), 2.43 MiB | 10.06 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [ ]:
import sys

In [ ]:
sys.path.insert(1, os.path.join(os.getcwd(), 'PPLM'))

#### PPLM con GPT2 base y bow de Uber

In [ ]:
from run_pplm import run_pplm_example

In [ ]:
seed = np.random.randint(low=0, high=10000, size=1)[0]

run_pplm_example(
    cond_text="In the last century",
    num_samples=5,
    seed = seed,
    bag_of_words='science',
    length=100,
    stepsize=0.02,
    sample=True,
    num_iterations=3,
    window_length=5,
    gamma=1.5,
    gm_scale=0.95,
    kl_scale=0.01,
    verbosity='quiet',
    colorama = True
)


= Prefix of sentence =
<|endoftext|>In the last century



/usr/local/lib/python3.7/dist-packages/transformers/modeling_gpt2.py:759: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


= Unperturbed generated text =
<|endoftext|>In the last century, many scientists and philosophers, especially those in the field of philosophy of mind theory have argued that there is an important relationship between human cognition and the mental capacities of the animal mind. The argument for this relationship has been made by both the classical natural philosophers and by the more recent thinkers who focus on animal cognitive processes, such as Dennett [4]. Although there are a number of problems with this argument [5], the central claim of this paper is that the relationship between cognition and animal cognitive capacities is

= Perturbed generated text 1 =
<|endoftext|>In the last century there has been an evolution in the way people live, eat and work.

This evolution in lifestyle has not only influenced the way the environment is generated; it has also influenced the way we live.

The evolution is in the way you eat, how you work, and the way you are affected by it.

As the ev

#### PPLM con modelo headlines y bows propios

In [ ]:
# versión modificada del pplm
from run_pplm_mod import run_pplm_example

In [ ]:
bow_path  = './bows/finance_bow.txt'

f = open(bow_path, 'r')
bow = f.read().split('\n')

In [ ]:
bow

['finance',
 'financial',
 'asset',
 'acquisition',
 'invest',
 'investor',
 'turnover',
 'ratio',
 'annuity',
 'balance',
 'market',
 'Brexit',
 'budget',
 'bond',
 'bussiness',
 'consume',
 'consumer',
 'consumption',
 'capital',
 'margin',
 'interest',
 'demand',
 'dividend',
 'economics',
 'economy',
 'earning',
 'equity',
 'externality',
 'entrepreneur',
 'enterprise',
 'fund',
 'economy',
 'economies',
 'stock',
 'fund',
 'money',
 'trade',
 'income',
 'globalization',
 'profit',
 'savings',
 'insurance',
 'interest',
 'liquidity',
 'liquidation',
 'monopoly',
 'oligopoly',
 'cost',
 'discount',
 'outsource',
 'profit',
 'tariff',
 'unemployment',
 'utility',
 'volatility']

In [ ]:
model_path = './model_headlines_news_old_transformers'
save_path = './generated_headlines/finance_headlines.txt'

bos = '<|endoftext|>'
eos = '<|EOS|>'

input_sentence = ' '

seed = np.random.randint(low=0, high=10000, size=1)[0]

run_pplm_example(
    pretrained_model = model_path,
    seed=seed,
    cond_text = input_sentence,
    num_samples=20,
    bag_of_words=bow_path,
    length=30,
    stepsize=0.025,
    sample=True,
    num_iterations=3,
    window_length=5,
    gamma=1.5,
    gm_scale=0.99,
    kl_scale=0.01,
    verbosity='quiet',
    colorama=True,
    stop_token = eos,
    save_path = save_path,
    append_to_path = True
)

= Prefix of sentence =
<|endoftext|> 



/usr/local/lib/python3.7/dist-packages/transformers/modeling_gpt2.py:759: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


= Unperturbed generated text =
The Tories can and should stop being so self-congratulatory 

= Perturbed generated text 1 =
How the UK's railways are responding to Brexit – a guide

= Perturbed generated text 2 =
Is there a better way to deal with the Trump-Trump trade war?

= Perturbed generated text 3 =
The Trump Administration Is Trying to ��Carpool It Back�� With ��Fake News��

= Perturbed generated text 4 =
A new kind of ��carpool seat�� for commuters: the cheap one

= Perturbed generated text 5 =
How the Trump presidency has affected the American economy. Here��s the full chart

= Perturbed generated text 6 =
The UK's trade deficit isn��t about trade, it��s about economics

= Perturbed generated text 7 =
The US is now a laughingstock of Europe – the only country in the world that can survive without a trade war

= Perturbed generated text 8 =
A new reality for the UK's energy companies

= Perturbed generated text 9 =
What a difference this week makes in how Trump is handling the 

In [ ]:
seed = np.random.randint(low=0, high=10000, size=1)[0]

bow_path  = './bows/politics_bow.txt'
save_path = './generated_headlines/politics_headlines.txt'

run_pplm_example(
    pretrained_model = model_path,
    seed=seed,
    cond_text = input_sentence,
    num_samples=20,
    bag_of_words=bow_path,
    length=30,
    stepsize=0.025,
    sample=True,
    num_iterations=3,
    window_length=5,
    gamma=1.5,
    gm_scale=0.99,
    kl_scale=0.01,
    verbosity='quiet',
    colorama=True,
    stop_token = eos,
    save_path = save_path,
    append_to_path = True
)

= Prefix of sentence =
<|endoftext|> 



/usr/local/lib/python3.7/dist-packages/transformers/modeling_gpt2.py:759: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


= Unperturbed generated text =
The Atlantic Politics & Policy Daily: “Trump’s ’Carpool Karaoke” 

= Perturbed generated text 1 =
How is China taking advantage of Trump��s trade war and trade war with North Korea?

= Perturbed generated text 2 =
How the media and politicians are using Trump��s victory to attack the truth of our immigration policy

= Perturbed generated text 3 =
Why is it important that you have a healthy diet for your kids?

= Perturbed generated text 4 =
The Trump administration is not a laughing matter

= Perturbed generated text 5 =
What is Brexit? The big question facing the US and China

= Perturbed generated text 6 =
Trump could easily get his party to the White House in a few days. Or he could get a little crazy

= Perturbed generated text 7 =
A decade of austerity would have made a mockery of democracy. But Brexit will not

= Perturbed generated text 8 =
The new ��Bachelor�� is a good show, but the show isn��t going to get off the ground

= Perturbed generated t

In [ ]:
seed = np.random.randint(low=0, high=10000, size=1)[0]

bow_path  = './bows/sports_bow.txt'
save_path = './generated_headlines/sports_headlines.txt'

print(seed)

run_pplm_example(
    pretrained_model = model_path,
    seed=seed,
    cond_text = input_sentence,
    num_samples=20,
    bag_of_words=bow_path,
    length=30,
    stepsize=0.07,
    sample=True,
    num_iterations=3,
    window_length=5,
    gamma=1.5,
    gm_scale=0.99,
    kl_scale=0.01,
    verbosity='quiet',
    colorama=True,
    stop_token = eos,
    save_path = save_path,
    append_to_path = True
)

8747
= Prefix of sentence =
<|endoftext|> 



/usr/local/lib/python3.7/dist-packages/transformers/modeling_gpt2.py:759: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


= Unperturbed generated text =
The world’s most expensive home is a luxury luxury 

= Perturbed generated text 1 =
��It��s a good deal��: US government seeks $20m in loans for oil and gas in Arctic region

= Perturbed generated text 2 =
Is this the best way to handle an Olympic final after a loss to Russia?

= Perturbed generated text 3 =
The Trump administration must stop trying to sell itself as good and wholesome at a time it��ll need to get it right

= Perturbed generated text 4 =
The best way to get your kids a good workout is to eat

= Perturbed generated text 5 =
A new way of eating is on the horizon – and it's killing obesity. Here are 10 tips to help you stay fit

= Perturbed generated text 6 =
The Yankees�� rotation just got better

= Perturbed generated text 7 =
The US economy could be in trouble in 2019

= Perturbed generated text 8 =
'They're all right': the US women's soccer team loses to England 1-0 (VIDEOS)

= Perturbed generated text 9 =
How a small but powerful tech c

Ejemplo de sobreajuste al bow

In [ ]:
seed = np.random.randint(low=0, high=10000, size=1)[0]

run_pplm_example(
    pretrained_model = model_path,
    seed=seed,
    cond_text = input_sentence,
    num_samples=20,
    bag_of_words=bow_path,
    length=30,
    stepsize=0.2,
    sample=True,
    num_iterations=3,
    window_length=5,
    gamma=1.5,
    gm_scale=0.99,
    kl_scale=0.01,
    verbosity='quiet',
    colorama=True,
    stop_token = eos
)

= Prefix of sentence =
<|endoftext|> 



/usr/local/lib/python3.7/dist-packages/transformers/modeling_gpt2.py:759: FutureWarning: The `past` argument is deprecated and will be removed in a future version, use `past_key_values` instead.
  FutureWarning,


= Unperturbed generated text =
Trump’s immigration ban could kill millions, but it could not kill millions of Americans 

= Perturbed generated text 1 =
The US Olympic hockey team just won the Olympic hockey medal race — but it��s still Olympic hockey

= Perturbed generated text 2 =
 Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo[0

= Perturbed generated text 3 =
'The best team in football' – as it happened

= Perturbed generated text 4 =
 Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo Ronaldo[0

= Perturbed generated text 5 =
Arsenal v Swansea City: Premier League Premier League Premier League Premier League Premier League Prem